In [1]:
# importing libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time

# import os
# import pandas as pd
# import matplotlib.pyplot as plt
# import datetime
# import argparse
# from bs4 import BeautifulSoup
# import requests

In [2]:
#url = "https://fr.trustpilot.com/review/eni.fr"

def page_info(url):

    def next_page():
        next_page_button_name = "pagination-button-next"
        next_button = driver.find_element(By.NAME, next_page_button_name)

        # we may have ElementClickInterceptedException that's why we are testing twice the instruction
        try:
            next_button.click()
        except:
            try:
                next_button.click()
            except ElementClickInterceptedException:
                pass

    def cookie_pass():
        cookie_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
        try:
            cookie_button.click()
        except:
            try:
                cookie_button.click()
            except ElementClickInterceptedException:
                pass

    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.get(url)
    cookie_pass()

    reviews_list = []
    max_page = 11       # Important : to have the first 10 pages, you need to do until 11 bc python excludes last
    for page in range(1, max_page):
        # Extract the review data from the current page
        reviews_elements = driver.find_elements(By.CLASS_NAME, "styles_reviewContentwrapper__zH_9M")

        reviews_page = list(
            map(
                lambda x: {
                    "text" : x.find_element(By.CLASS_NAME, "styles_reviewContent__0Q2Tg").text,
                    "grade" : x.find_element(By.CLASS_NAME, "styles_reviewHeader__iU9Px").get_attribute("data-service-review-rating"),
                    "date": x.find_element(By.CLASS_NAME, "styles_reviewHeader__iU9Px").text,
                },
                reviews_elements,
            )
        )
        reviews_list.extend(reviews_page)
        
        next_page()
        time.sleep(2)  # for the page to load
        

    # Close the webdriver
    # driver.quit()

    return reviews_list

# 1. Scraping Trustpilot for ENI

In [3]:
web_data = page_info(url="https://fr.trustpilot.com/review/eni.fr")

#### Save data as CSV

In [7]:
print(web_data[0])
print(web_data[0].keys())

{'text': "Catastrophe énergétique\nPremière souscription avec eux en 2021 :\npour la première année aucune régularisation en 2022 à faire montant annuel 880 euros\ndeuxième année 2022 facture de régularisation en 2023 de 1137 euros pour un montant total de consommation de 1880 euros.\nJ'ai essayé de vérifier ma consommation sur mon espace client, je remarque que les données affichées ne sont pas cohérentes (dans un mois de l'année j'étais en vacances, il n'y avait personne à la maison et tout les appareils était éteints).\n\nJ'ai appelé le service client le 01/02/2023 pour avoir des explications, la conseillère me dis qu'elle n'a pas plus de visibilité sur mon dossier, elle a demandé l'intervention d'un autre service qui au jour d'aujourd'hui ne m'a pas rappelé.\n\nUne catastrophe totale, je regrette d'avoir choisi ENI\nDate de l'expérience: 03 février 2023", 'grade': '1', 'date': 'Il y a 5 heures'}
dict_keys(['text', 'grade', 'date'])


In [9]:
len(web_data)

200

In [22]:
import pandas as pd
df = pd.DataFrame(columns=web_data[0].keys())

for i, comment in enumerate(web_data):
    df.loc[i, "text"] = comment["text"]
    df.loc[i, "grade"] = comment["grade"]
    df.loc[i, "date"] = comment["date"]
    
df

,text,grade,date
0,Catastrophe énergétique\nPremière souscription...,1,Il y a 5 heures
1,Augmentation du prix de kWh X3\nAugmentation d...,1,ll y a 4 jours
2,BRAQUEURS ENI\nSimplement : échéancier 2022 : ...,1,ll y a 6 jours
3,Facture démentielle.\nFacture démentielle.\nLe...,1,ll y a 4 jours
4,A fuir !\nAyant un relevé tous les deux mois p...,1,26 janv. 2023
...,...,...,...
195,Belle erreur d'avoir été chez ENI\nJe ne suis ...,1,26 déc. 2021
196,"La pire entreprise\nLa pire entreprise, juste ...",1,7 févr. 2022
197,J'ai résilié mon contrat chez Eni le 30…\nJ'ai...,1,13 oct. 2021
198,Service client catastrophique et après une ven...,1,5 sept. 2022


In [25]:
import os 
os.getcwd()

'C:\\Users\\Dell\\MASTER COURSES\\CapGemini Datacamp\\Session 1\\HW1'

In [28]:
df.to_csv("eni_reviews.csv")

In [37]:
#Save data as text file
file = open('capgemini.txt' , 'w')
for page in web_data:
    print("\n\n")
    for k, v in page.items():
        try:
            file.write(k + ":" + v + "\n")
        except:
            continue

file.close()    

# 2. Scraping Trustpilot for TotalEnergies

In [30]:
web_data2 = page_info(url="https://fr.trustpilot.com/review/totalenergies.fr")

In [35]:
df = pd.DataFrame(columns=web_data2[0].keys())

for i, comment in enumerate(web_data2):
    df.loc[i, "text"] = comment["text"]
    df.loc[i, "grade"] = comment["grade"]
    df.loc[i, "date"] = comment["date"]
    
df.to_csv("TotalEnergies_reviews.csv")

In [38]:
file = open('TotalEnergies.txt' , 'w')
for page in web_data:
    print("\n\n")
    for k, v in page.items():
        try:
            file.write(k + ":" + v + "\n")
        except:
            continue

file.close()    

In [34]:
with open('TotalEnergies.txt') as f:
    lines = f.readlines()
lines

['text:A fuir !\n',
 "Ayant un relevé tous les deux mois pour ma consommation réelle d'électricité, je passe d'un montant de 170€ a un montant de 460€ pour casi la même consommation de kwh.\n",
 'Sans mails, sans courriers de leur part pour prévenir.\n',
 "C'est une honte !.\n",
 'Fuyez ENI.\n',
 "Date de l'expérience: 08 janvier 2023\n",
 'grade:1\n',
 'date:Il y a un jour\n',
 'text:Escroquerie ENI\n',
 "Je rejoins les derniers avis ci-dessous, ENI a multiplié par 5 le prix de son kwH sans aucun signalement convenable. Je pense que quand on vous réclame 6000€ d'électricité par an vous pouvez envoyer des recommandés pour prévenir vos clients... Pour information; la réponse du service client a été de me dire que ENI ne propose plus d'offre électricité seul, il cherche donc à nous dégager par des méthodes minables. Pour ma part c'est réussi mais je ne laisserais pas passer. Je vous invite tous à écrire au médiateur de l'énergie, site du gouvernent fait pour cela : chercher médiateur de 